### ML with PySpark
+ Classify/Predict 

In [1]:
# Load our Pkgs

import sys; sys.path.append("../")
from DataPreprocessing.DataPreprocessing import read_data
from pyspark import SparkContext

In [2]:
sc = SparkContext(master='local')

In [3]:
# Load Pkgs 
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType,BooleanType,DateType
from pyspark.sql.functions import col

In [4]:
# Spark
spark = SparkSession.builder.appName("MLwithSpark").getOrCreate()

In [5]:
# Load our dataset
df= read_data(spark,file_name='train')

In [6]:
# Rearrange
df = df.select('App Id', 'App Name', 'Category', 'Rating', 'Rating Count', 'Installs', 'Minimum Installs', 'Maximum Installs', 'Free', 'Price', 'Size', 'Minimum Android', 'Developer Id', 'Developer Email', 'Released', 'Last Updated', 'Content Rating', 'Ad Supported', 'In App Purchases', 'Editors Choice')

In [7]:
#cast some columns to numeric values not strings
df = df.na.drop(how='any', subset=['Rating', 'Rating Count', 'Minimum Installs', 'Maximum Installs', 'Free', 'Price', 'Content Rating', 'Category'])

In [8]:
import pyspark.ml

In [9]:
# Load ML Pkgs
from pyspark.ml.feature import VectorAssembler,StringIndexer

In [10]:
# Convert the string into numerical code
# label encoding
categoryEncoder = StringIndexer(inputCol='Category',outputCol='Category_enc', handleInvalid='skip').fit(df)
df = categoryEncoder.transform(df)

categoryEncoder = StringIndexer(inputCol='Free',outputCol='Free_enc', handleInvalid='skip').fit(df)
df = categoryEncoder.transform(df)

categoryEncoder = StringIndexer(inputCol='Content Rating',outputCol='Content Rating_enc',  handleInvalid='skip').fit(df)
df = categoryEncoder.transform(df)

categoryEncoder = StringIndexer(inputCol='Ad Supported',outputCol='Ad Supported_enc', handleInvalid='skip').fit(df)
df = categoryEncoder.transform(df)

categoryEncoder = StringIndexer(inputCol='In App Purchases',outputCol='In App Purchases_enc', handleInvalid='skip').fit(df)
df = categoryEncoder.transform(df)

categoryEncoder = StringIndexer(inputCol='Editors Choice',outputCol='Editors Choice_enc', handleInvalid='skip').fit(df)
df = categoryEncoder.transform(df)

df.na.fill(value=0.0)



DataFrame[App Id: string, App Name: string, Category: string, Rating: float, Rating Count: int, Installs: string, Minimum Installs: int, Maximum Installs: int, Free: string, Price: float, Size: string, Minimum Android: string, Developer Id: string, Developer Email: string, Released: string, Last Updated: string, Content Rating: string, Ad Supported: string, In App Purchases: string, Editors Choice: string, Category_enc: double, Free_enc: double, Content Rating_enc: double, Ad Supported_enc: double, In App Purchases_enc: double, Editors Choice_enc: double]

In [11]:
# encoded column back to String
from pyspark.ml.feature import IndexToString
converter = IndexToString(inputCol='Category_enc',outputCol='orig_category')
converted_df = converter.transform(df)

### Features


In [12]:
df2 = df.select('Rating Count','Minimum Installs', 'Maximum Installs', 'Free_enc', 'Price', 'Content Rating_enc', 'Ad Supported_enc', 'In App Purchases_enc', 'Editors Choice_enc', 'Category_enc', 'Rating')

In [13]:
required_features = df2.columns[:-1]      #all except the target variable 'Rating' which is at the last index
print(required_features)

['Rating Count', 'Minimum Installs', 'Maximum Installs', 'Free_enc', 'Price', 'Content Rating_enc', 'Ad Supported_enc', 'In App Purchases_enc', 'Editors Choice_enc', 'Category_enc']


In [14]:
# VectorAsm
vec_assembler = VectorAssembler(inputCols=required_features,outputCol='Features')

In [15]:
vec_df = vec_assembler.setHandleInvalid("skip").transform(df2)

In [16]:
vec_df.na.drop(how='any',subset='Features')

DataFrame[Rating Count: int, Minimum Installs: int, Maximum Installs: int, Free_enc: double, Price: float, Content Rating_enc: double, Ad Supported_enc: double, In App Purchases_enc: double, Editors Choice_enc: double, Category_enc: double, Rating: float, Features: vector]

### Train,Test Split

In [17]:
train_df,test_df = vec_df.select(['Features', 'Rating']).randomSplit([0.7,0.3])

#### Model Building
+ Pyspark.ml: DataFrame
+ Pyspark.mllib: RDD /Legacy

In [18]:
from pyspark.ml.regression import LinearRegression

In [19]:
# Regr Model
lr = LinearRegression(featuresCol='Features',labelCol='Rating')

In [20]:
lr_model = lr.fit(train_df)

#just some training metrics
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 1.874600
r2: 0.217931


In [21]:
y_pred = lr_model.transform(test_df)

#notice another column called 'prediction' is automatically made
y_pred.show(5)

+----------+------+------------------+
|  Features|Rating|        prediction|
+----------+------+------------------+
|(10,[],[])|   0.0|1.2354008672863137|
|(10,[],[])|   0.0|1.2354008672863137|
|(10,[],[])|   0.0|1.2354008672863137|
|(10,[],[])|   0.0|1.2354008672863137|
|(10,[],[])|   0.0|1.2354008672863137|
+----------+------+------------------+
only showing top 5 rows



#### Model Evaluation

In [22]:
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="Rating",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(y_pred))

R Squared (R2) on test data = 0.189946


In [23]:
test_result = lr_model.evaluate(test_df)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

Root Mean Squared Error (RMSE) on test data = 1.90788


#### Decision Tree Regression

In [24]:
from pyspark.ml.regression import DecisionTreeRegressor
dt = DecisionTreeRegressor(featuresCol ='Features', labelCol = 'Rating', maxBins=48)   #setting a larger maxbins(default=32) is an ad-hoc step just to solve an error I dont understand :)
dt_model = dt.fit(train_df)
dt_predictions = dt_model.transform(test_df)
dt_evaluator = RegressionEvaluator(
    labelCol="Rating", predictionCol="prediction", metricName="rmse")
rmse = dt_evaluator.evaluate(dt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

IllegalArgumentException: requirement failed: DecisionTree requires maxBins (= 48) to be at least as large as the number of values in each categorical feature, but categorical feature 9 has 53 values. Consider removing this and other categorical features with a large number of values, or add more training examples.

In [ ]:
#Feature Importance
dt_model.featureImportances

#notice that Feature at index 0 has higher importance, this feature is the 'Rating Count'

SparseVector(10, {0: 0.9917, 1: 0.0, 2: 0.0061, 3: 0.0001, 4: 0.0, 6: 0.0003, 9: 0.0018})

### Gradient-Boosted Decision Tree Regressor

In [ ]:
from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(featuresCol = 'Features', labelCol = 'Rating', maxIter=10, maxBins=48) #same as above, maxbins is set just to avoid an error I dont understand
# maxIter is just a hyperparameter we put by hand
gbt_model = gbt.fit(train_df)
gbt_predictions = gbt_model.transform(test_df)
gbt_predictions.select('prediction', 'Rating', 'Features').show(5)

+--------------------+------+----------+
|          prediction|Rating|  Features|
+--------------------+------+----------+
|-0.00190186993011...|   0.0|(10,[],[])|
|-0.00190186993011...|   0.0|(10,[],[])|
|-0.00190186993011...|   0.0|(10,[],[])|
|-0.00190186993011...|   0.0|(10,[],[])|
|-0.00190186993011...|   0.0|(10,[],[])|
+--------------------+------+----------+
only showing top 5 rows



In [ ]:
gbt_evaluator = RegressionEvaluator(
    labelCol="Rating", predictionCol="prediction", metricName="rmse")
rmse = gbt_evaluator.evaluate(gbt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 0.45322


### Saving Model


In [ ]:
#this is giving me errors I still cannot resolve

from pyspark.ml.pipeline import PipelineModel
gbt_model.save("D:/CLASSSWORKKK/BD/BD_PROJECT/GBT_Model")
#to overwrite an existing model
gbt_model.write().overwrite().save("D:/CLASSSWORKKK/BD/BD_PROJECT/GBT_Model")

Py4JJavaError: An error occurred while calling o913.save.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java

### Loading Model

In [ ]:
from pyspark.ml.pipeline import PipelineModel
LoadedModel = PipelineModel.load("./GBT_Model")

Py4JJavaError: An error occurred while calling o1696.partitions.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/d:/CLASSSWORKKK/BD/BD_PROJECT/GBT_Model/metadata
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:304)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:244)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:332)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:208)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:291)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:287)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:291)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:287)
	at org.apache.spark.api.java.JavaRDDLike.parti